In [1]:
import os, pandas as pd, scanpy as sc, numpy as np, warnings
from geneformer import TranscriptomeTokenizer, EmbExtractor
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
min_cells: 5; min_genes: 500; min_counts: 1000

In [2]:
def preprocess(adata, top_n_cells=100, min_cells=5, min_genes=500, min_counts=1000):
    # Sanity Check
    subset = adata.X[:, :top_n_cells].toarray()
    non_negative = np.all(subset >= 0)
    integer_values = np.all(subset.astype(int) == subset)
    assert non_negative and integer_values

    # Quality Control
    sc.pp.filter_cells(adata, min_counts=min_counts)
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    print("=" * 77)
    print(rf"{adata.n_vars} genes x {adata.n_obs} cells after quality control.")
    print("=" * 77)
    return adata

### Heart -> Eye -> gut_fetal -> lung -> skin -> lung_fetal_organoid -> lung_fetal_donor -> brain -> breast

In [21]:
dataset = "brain"
DATA_DIR = "/home/wangh256/G-scIB_dev/data/%s/" % dataset

# adata = sc.read_h5ad(rf"{DATA_DIR}/emb.h5ad")
adata = sc.read_h5ad(rf"{DATA_DIR}/local.h5ad")
# adata = sc.read_h5ad(rf"{DATA_DIR}/organoid.h5ad")
# adata = sc.read_h5ad(rf"{DATA_DIR}/donor.h5ad")
adata

AnnData object with n_obs × n_vars = 888263 × 59357
    obs: 'ROIGroup', 'ROIGroupCoarse', 'ROIGroupFine', 'roi', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'Biotype', 'Chromosome', 'End', 'Gene', 'Start', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'batch_condition', 'schema_version', 'title'
    obsm: 'X_UMAP', 'X_tSNE'

In [22]:
# adata.X = adata.raw.X
# del adata.raw
adata = preprocess(adata, min_genes=1000) # only for brain, my bad

42079 genes x 839273 cells after quality control.


In [20]:
for min_cells in [5, 10, 100]:
    for min_genes in [500, 1000]:
        for min_counts in [1000, 5000]:
            print(rf"min_cells: {min_cells}; min_genes: {min_genes}; min_counts: {min_counts}")
            adata = sc.read_h5ad(rf"{DATA_DIR}/local.h5ad")
            _ = preprocess(adata, min_cells=min_cells, min_genes=min_genes, min_counts=min_counts)
            print("\n\n")

min_cells: 5; min_genes: 500; min_counts: 1000
42135 genes x 885790 cells after quality control.



min_cells: 5; min_genes: 500; min_counts: 5000
40843 genes x 345182 cells after quality control.



min_cells: 5; min_genes: 1000; min_counts: 1000
42079 genes x 839273 cells after quality control.



min_cells: 5; min_genes: 1000; min_counts: 5000
40843 genes x 345182 cells after quality control.



min_cells: 10; min_genes: 500; min_counts: 1000
40114 genes x 885790 cells after quality control.



min_cells: 10; min_genes: 500; min_counts: 5000
38772 genes x 345182 cells after quality control.



min_cells: 10; min_genes: 1000; min_counts: 1000
40068 genes x 839273 cells after quality control.



min_cells: 10; min_genes: 1000; min_counts: 5000
38772 genes x 345182 cells after quality control.



min_cells: 100; min_genes: 500; min_counts: 1000
32851 genes x 885790 cells after quality control.



min_cells: 100; min_genes: 500; min_counts: 5000


KeyboardInterrupt: 

In [23]:
adata.var

,Biotype,Chromosome,End,Gene,Start,feature_is_filtered,feature_name,feature_reference,feature_biotype,n_cells
ensembl_ids,,,,,,,,,,
ENSG00000000003,n/a,chrX,100639991,TSPAN6,100627108,False,TSPAN6,NCBITaxon:9606,gene,30075
ENSG00000000005,n/a,chrX,100599885,TNMD,100584936,False,TNMD,NCBITaxon:9606,gene,485
ENSG00000000419,n/a,chr20,50958555,DPM1,50934867,False,DPM1,NCBITaxon:9606,gene,129625
ENSG00000000457,n/a,chr1,169894267,SCYL3,169849631,False,SCYL3,NCBITaxon:9606,gene,97314
ENSG00000000460,n/a,chr1,169854080,C1orf112,169662007,False,C1orf112,NCBITaxon:9606,gene,82343
...,...,...,...,...,...,...,...,...,...,...
ENSG00000288611,n/a,chr8,52943734,NPBWR1,52939182,False,NPBWR1,NCBITaxon:9606,gene,144
ENSG00000288612,n/a,chr6,3027426,AL133351.4,3023472,False,RP1-90J20.15,NCBITaxon:9606,gene,15576
ENSG00000288632,n/a,chr16,29613717,AC133555.6,29442917,False,RP11-345J4.11,NCBITaxon:9606,gene,77888


In [24]:
adata.var['ensembl_id'] = adata.var.index.tolist()

In [25]:
os.makedirs(rf"{DATA_DIR}/Geneformer", exist_ok=True)
os.makedirs(rf"{DATA_DIR}/GeneformerRaw", exist_ok=True)
adata.write_h5ad(rf"{DATA_DIR}/GeneformerRaw/local.h5ad", compression="gzip")

tk = TranscriptomeTokenizer({}, nproc=16)
tk.tokenize_data(rf"{DATA_DIR}/GeneformerRaw", 
                 rf"{DATA_DIR}/Geneformer", 
                 dataset, 
                 file_format="h5ad")

Tokenizing /home/wangh256/G-scIB_dev/data/brain/GeneformerRaw/local.h5ad
/home/wangh256/G-scIB_dev/data/brain/GeneformerRaw/local.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map (num_proc=16):   0%|          | 0/839273 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/839273 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/839273 [00:00<?, ? examples/s]

In [ ]:
embex = EmbExtractor(model_type="Pretrained",
                     num_classes=0,
                     max_ncells=None,
                     forward_batch_size=128,
                     nproc=16)

embs = embex.extract_embs("/home/wangh256/GeneDataEngine_dev/Geneformer-main/geneformer-12L-30M",
                          rf"{DATA_DIR}/Geneformer/Eye.dataset",
                          rf"{DATA_DIR}/",
                          "Geneformer")

In [5]:
bdata = sc.read_h5ad(rf"{DATA_DIR}/emb.h5ad")
bdata.obsm["Geneformer"] = embs.values

In [6]:
bdata.write_h5ad(rf"{DATA_DIR}/emb.h5ad", compression="gzip")

In [7]:
bdata.obsm

AxisArrays with keys: Harmony, Islander, Islander_UMAP, Scanorama, X_bbknn, X_pca, X_scANVI, X_scVI, X_tsne, X_umap, scPoli, Geneformer

In [10]:
bdata.obsm["Geneformer"].shape

(380078, 512)